# on improving lowdin with kpm

## imports here

In [ ]:
import kwant
import numpy as np
import scipy.linalg as la
import scipy.sparse as sla

import itertools

import matplotlib.pyplot as plt
%matplotlib inline

## prepare random Hamiltonian

In [ ]:
def H0_random(nA=4, nB=100, gap=1, epsilonA=0.2, epsilonB=10):
    """Generate random Hamiltonian with quasi-degenerate states."""
    energiesA = epsilonA * np.random.random(nA) - epsilonA / 2
    
    energiesB = epsilonB * np.random.random(nB) - epsilonB / 2
    energiesB = energiesB[np.abs(energiesB) > gap/2]

    energies = np.append(energiesA, energiesB)
    U = kwant.rmt.circular(len(energies))
    
    return np.diag(energies)#U.transpose().conjugate() @ np.diag(energies) @ U


def H1_random(n, v=0.1):
    return kwant.rmt.gaussian(n, v=v)

In [ ]:
alphas = np.logspace(-8, 0, 100)
np.random.seed(0)

H0 = H0_random()        # This is H_0
H1 = H1_random(len(H0)) # This is perturbation (H')


## exact solution

$H(\alpha) = H_0 + \alpha H^\prime$

In [ ]:
%%time

window = (-.25, +.25)
ev, evec = la.eigh(H0)

indices = [i for (i, e) in enumerate(ev) if window[0] < e < window[1]]
n = len(indices)

In [ ]:
%%time

energies = []
for alpha in alphas:
    e = la.eigvalsh(H0 + alpha * H1)
    energies.append(e)
energies = np.array(energies)

In [ ]:
energies.shape

In [ ]:
plt.plot(alphas, np.array(energies), 'C0');
plt.ylim(-1, 1)    

## explicit lowdin implementation

$
H^{(0)}_{ij} = (H_0)_{i,j}
$

$
H^{(1)}_{ij} = (H^\prime)_{i,j}
$

$
H^{(2)}_{ij} = \frac{1}{2} \sum_m \left[(H^\prime)_{i,m} (H^\prime)_{m, j} \times \left(\frac{1}{E_i-E_m} + \frac{1}{E_j - E_m} \right) \right]
$

In [ ]:
%%time

def triproduct(left, matrix, right):
    return left.conjugate() @ matrix @ right

H_eff_1 = []
H_eff_2 = []

# iterate over states in group A
for i, j in itertools.product(indices, indices):
    
    # calculate 1-st order perturbation
    H_eff_1.append(triproduct(evec[:, i], H1, evec[:, j]))
    
    element = 0
    # iterate over states in group B
    for m in range(len(ev)):

        if m in indices:
            continue

        # This calculates H'_{im} H'_{mj} x (1 / (Ei - Em) + 1 / (Ej / Em))
        v1 = triproduct(evec[:, i], H1, evec[:, m])
        v2 = triproduct(evec[:, m], H1, evec[:, j])
        element += v1 * v2 * (1 / (ev[i] - ev[m]) + 1 / (ev[j] - ev[m]))
        
    H_eff_2.append(0.5 * element)
    
    
H_eff_0 = np.diag(ev[indices])    
H_eff_1 = np.array(H_eff_1).reshape(n, n)
H_eff_2 = np.array(H_eff_2).reshape(n, n)

In [ ]:
H_eff_0

In [ ]:
H_eff_1

In [ ]:
H_eff_2

In [ ]:
# only h1_eff
energies_effective_1 = []
# up to h2_eff
energies_effective_2 = []

for alpha in alphas:
    H_eff = (H_eff_0 +
             alpha * H_eff_1 +
             0**2 * H_eff_2
            )
    
    e = la.eigvalsh(H_eff)
    energies_effective_1.append(e)
    H_eff = (H_eff_0 +
             alpha * H_eff_1 +
             alpha**2 * H_eff_2
            )
    
    e = la.eigvalsh(H_eff)
    energies_effective_2.append(e)

In [ ]:
# first and second order

fig = plt.figure(figsize=(4,4))

plt.plot(alphas, np.array(energies), 'C0');
plt.plot(alphas, np.array(energies_effective_1), 'C3-');
plt.plot(alphas, np.array(energies_effective_2), 'C8-');
plt.ylim(-.35, .35)


In [ ]:
import holoviews as hv
hv.notebook_extension()

In [ ]:
%%opts Overlay [logx=True logy=True]

(
    hv.Path((alphas, np.abs(energies[:,indices]-energies_effective_1))) *
    hv.Path((alphas, np.abs(energies[:,indices]-energies_effective_2)))
)[1e-8:,1e-16:]

# kpm improvement (below unfinished)

Goal is to optimize $H^{(2)}$ using KPM.

My rough understanding from the discussion comes here:

$
H^{(2)}_{ij} \rightarrow \frac{1}{2} <i | H^\prime  \left[\sum_m |m > < m| \left( \frac{1}{E_i-E_m} + \frac{1}{E_j - E_m}\right) \right] H^\prime |j>
$

Now with

$
\sum_m |m > < m| \frac{1}{E_i-E_m} = \sum_m |m > < m| \frac{1}{E_i-H_0} = P_B \frac{1}{E_i-E_m} = f(E_i, H_0)
$

we transform further into
$
H^{(2)}_{ij} \rightarrow \frac{1}{2} <i | H^\prime  \left[ f(E_i, H_0) + f(E_j, H_0) \right] H^\prime |j>
$

In [ ]:
%%time

window = (-.25, +.25)
ev, evec = la.eigh(H0)

indices = [i for (i, e) in enumerate(ev) if window[0] < e < window[1]]
n = len(indices)

In [ ]:
%%time

def triproduct(left, matrix, right):
    return left.conjugate() @ matrix @ right


def proj(vec, subspace):
    P_A = subspace @ subspace.T.conj()
    return vec -  P_A @ vec


H_eff_1 = []
H_eff_2 = []

# iterate over states in group A
for i, j in itertools.product(indices, indices):
    
    # calculate 1-st order perturbation
    H_eff_1.append(triproduct(evec[:, i], H1, evec[:, j]))    
    
H_eff_0 = np.diag(ev[indices])    
H_eff_1 = np.array(H_eff_1).reshape(n, n)

In [ ]:
from codes.kpm_funcs import build_perturbation

In [ ]:
kpm_params = dict(num_moments=1000)
H_eff_2 = build_perturbation(ev[indices], evec[:,indices], H0, H1, kpm_params=kpm_params)

In [ ]:
# only h1_eff
energies_effective_1 = []
# up to h2_eff
energies_effective_2 = []

for alpha in alphas:
    H_eff = (H_eff_0 +
             alpha * H_eff_1 +
             0**2 * H_eff_2
            )
    
    e = la.eigvalsh(H_eff)
    energies_effective_1.append(e)
    H_eff = (H_eff_0 +
             alpha * H_eff_1 +
             alpha**2 * H_eff_2
            )
    
    e = la.eigvalsh(H_eff)
    energies_effective_2.append(e)

In [ ]:
# first and second order

fig = plt.figure(figsize=(4,4))

plt.plot(alphas, np.array(energies), 'C0');
plt.plot(alphas, np.array(energies_effective_1), 'C3-');
plt.plot(alphas, np.array(energies_effective_2), 'C8-');
plt.ylim(-.35, .35)


In [ ]:
%%opts Overlay [logx=True logy=True]

(
    hv.Path((alphas, np.abs(energies[:,indices]-energies_effective_1))) *
    hv.Path((alphas, np.abs(energies[:,indices]-energies_effective_2)))
)[1e-8:,1e-16:]